# 🚀 **AI-Pulse tutorial: Start predicting with NICL**



## **Import required libraries**

In [ ]:
import os

import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

!pip install skrub
from skrub import TableVectorizer

!pip install neuralk
from neuralk import OnPremiseClassifier

# for comparison
!pip install xgboost
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder



##**Log in to server**

## **Download OpenML dataset**
For this tutorial we use the **GesturePhaseSegmentationProcessed** dataset from OpenML.


It contains numerical motion features extracted from 7 videos of people gesturing, including velocities and accelerations of both hands and wrists along x, y, and z coordinates, as well as scalar velocities. In summary:

- 9873 samples, 32 numerical features
- Each sample labeled with a gesture phase (D, P, S, H, R)
- Goal: Predict the gesture phase based on hand motion features.


In [ ]:
from sklearn.datasets import fetch_openml

# Load data
df = fetch_openml(data_id=4538)

X, y = df.data, df.target

print(X.shape, y.shape)

(9873, 32) (9873,)


In [ ]:
df.data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32
count,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000,9873.000000,9.873000e+03,...,9873.000000,9.873000e+03,9873.000000,9873.000000,9873.000000,9873.000000,9.873000e+03,9.873000e+03,9.873000e+03,9.873000e+03
mean,0.000016,0.000020,0.000013,-0.000002,-0.000006,0.000015,0.000014,0.000001,0.000015,1.954562e-06,...,-0.000002,-2.187289e-07,0.008564,0.011116,0.007090,0.009083,1.478127e-03,1.736036e-03,1.273790e-03,1.455456e-03
std,0.007183,0.009816,0.007082,0.008988,0.012803,0.007560,0.005993,0.008202,0.005991,7.182825e-03,...,0.001852,9.539442e-04,0.011169,0.013351,0.009423,0.011077,2.413924e-03,2.698434e-03,2.047390e-03,2.190217e-03
min,-0.072620,-0.079209,-0.084849,-0.095705,-0.107258,-0.099784,-0.053584,-0.067566,-0.072807,-7.287716e-02,...,-0.026311,-1.498854e-02,0.000019,0.000011,0.000012,0.000005,4.500000e-07,2.900000e-07,3.300000e-07,2.500000e-07
25%,-0.001743,-0.001685,-0.000415,-0.002148,-0.002852,-0.000617,-0.001447,-0.001389,-0.000375,-1.809190e-03,...,-0.000293,-6.886000e-05,0.001159,0.001625,0.000972,0.001308,1.377600e-04,1.994200e-04,1.148900e-04,1.705000e-04
50%,0.000010,0.000029,-0.000004,-0.000009,-0.000029,-0.000013,0.000004,0.000006,-0.000007,-9.900000e-07,...,0.000001,9.500000e-07,0.004132,0.006227,0.003412,0.005014,5.391000e-04,8.184300e-04,4.718800e-04,6.603500e-04
75%,0.001675,0.001980,0.000367,0.002485,0.002885,0.000515,0.001493,0.001611,0.000319,1.989900e-03,...,0.000306,8.390000e-05,0.011728,0.015775,0.009556,0.012609,1.841250e-03,2.283890e-03,1.601050e-03,1.941810e-03
max,0.073446,0.071600,0.091322,0.071854,0.132058,0.072152,0.069779,0.055297,0.064984,5.523441e-02,...,0.062295,1.053080e-02,0.106504,0.154135,0.077625,0.124277,4.262203e-02,9.232396e-02,3.933604e-02,7.354483e-02


In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of classes: {len(np.unique(y))}")

Training set size: 7898
Test set size: 1975
Number of features: 32
Number of classes: 5


In [ ]:
vec = TableVectorizer(
    numeric=Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]),
    low_cardinality=Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    ])
)

X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)

print(f"  - Training shape: {X_train.shape}")
print(f"  - Test shape: {X_test.shape}")


  - Training shape: (7898, 32)
  - Test shape: (1975, 32)


## **Start making predictions with NICL**



Behind Neuralk's Classifier() is NICL, our foundation model pre-trained on millions of synthetic tabular datasets.

Instead of training a model from scratch on your data, NICL leverages an In-Context Learning (ICL) architecture that uses your training data as context, allowing you to make predictions instantly without traditional model training on your test set.

You can learn more about it here.

In [ ]:
classifier = OnPremiseClassifier(host="http://51.159.172.205:5764")

classifier.fit(X_train, y_train)
print("✓ Classifier fitted successfully - In our case, this only means saving the X_train and y_train in the classifier object for the inference")

# Make predictions
predictions = classifier.predict(X_test)

acc = accuracy_score(y_test, predictions)
print(f"✓ Accuracy: {acc:.3f}")

✓ Classifier fitted successfully - In our case, this only means saving the X_train and y_train in the classifier object for the inference
✓ Accuracy: 0.784


## **Let's compare with a traditional model (XGBoost)**



In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# XGBoost requires integer class labels (0..num_classes-1)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)  # use same encoder

# Traditional models like XGBoost need explicit training and do not benefit
# from millions of priors learned across datasets.
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    tree_method="hist",
    use_label_encoder=False,
    enable_categorical=True
)

xgb_model.fit(X_train, y_train_encoded)

pred = xgb_model.predict(X_test)
acc_xgb = accuracy_score(y_test_encoded, pred)
print("XGBoost Accuracy:", acc_xgb)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [11:30:48] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Accuracy: 0.6855696202531646


## 🔎 **Observations**




In this example, NICL outperforms XGBoost, without any model training on the target data thanks to a powerful pre-training strategy and ICL architecture.

On real-world tabular datasets, NICL can achieve more than **30% higher performance** compared to classic ML methods. You can explore the full benchmark [here](https://dashboard.neuralk-ai.com/#/industrial/product-categorization).

**When to use**

NICL performs best on datasets with up to 250 features and 15,000 samples, offering reliable and consistent performance out of the box.

For larger problems, NICL can scale to 1 million samples and 500 features, though performance may vary depending on input complexity.

NICL is ideal when you:
- Need strong baseline performance without hyper-parameter tuning.
- Want a unified approach to handle mixed feature types.
- Are exploring new datasets and want fast iteration.
- Prefer interpretability and flexible prompting over black-box optimisation.

[Read the docs here.
](https://docs.neuralk-ai.com/docs/intro)